<a href="https://colab.research.google.com/github/krcpr007/iris_encryption-/blob/main/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from PIL import Image
import os
import numpy as np
image_folder = '/content/gdrive/MyDrive/iris/train/eye'
images = []
for filename in os.listdir(image_folder):
    img = Image.open(os.path.join(image_folder, filename)).convert('L') # convert to grayscale
    img = img.resize((512, 512)) # resize image to 64x64 pixels
    images.append(img)

In [ ]:
from sklearn.preprocessing import StandardScaler

# convert images to numpy arrays
X = []
for img in images:
    X.append(np.array(img).flatten())
X = np.array(X)

# normalize dataset
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
# print(X_normalized)

In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf
import keras

In [ ]:

# define CNN model
model = models.Sequential([
    layers.Reshape((512, 512, 1), input_shape=(262144,)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# compile model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# train model
y = np.array([0] * 50 + [1] * 50 + [2] * 50) # set target labels
y_onehot = tf.one_hot(y, depth=3) # one-hot encode target labels
X_normalized = X_normalized[:150]
model.fit(X_normalized, y_onehot, epochs=10)


test_loss, test_acc = model.evaluate(X_normalized, y_onehot)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/10
5/5 [==============================] - 92s 17s/step - loss: 12.6062 - accuracy: 0.4067
Epoch 2/10
5/5 [==============================] - 87s 17s/step - loss: 1.1194 - accuracy: 0.4133
Epoch 3/10
5/5 [==============================] - 94s 19s/step - loss: 0.9864 - accuracy: 0.5467
Epoch 4/10
5/5 [==============================] - 88s 17s/step - loss: 0.7890 - accuracy: 0.7000
Epoch 5/10
5/5 [==============================] - 87s 17s/step - loss: 0.4944 - accuracy: 0.8267
Epoch 6/10
5/5 [==============================] - 97s 18s/step - loss: 0.2213 - accuracy: 0.9600
Epoch 7/10
5/5 [==============================] - 88s 18s/step - loss: 0.0520 - accuracy: 0.9933
Epoch 8/10
5/5 [==============================] - 91s 17s/step - loss: 0.0383 - accuracy: 0.9933
Epoch 9/10
5/5 [==============================] - 88s 18s/step - loss: 0.0048 - accuracy: 1.0000
Epoch 10/10
5/5 [==============================] - 21s 4s/step - loss: 0.0017 - accuracy: 1.0000
Test loss: 0.0017468675505369

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# extract feature vector V1
V1 = model.predict(X_normalized)
# print(V1.shape)

5/5 [==============================] - 23s 4s/step


In [ ]:
!pip install reedsolo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bchlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bchlib: filename=bchlib-0.14.0-cp310-cp310-linux_x86_64.whl size=70699 sha256=0bc056f39c01cd58b4c209c6cfe1ad51a4c5d76070d5633e691899b2741514eb
  Stored in directory: /root/.cache/pip/wheels/8e/6f/cc/5eab719b09161264635161f48773df088422a94ae61c985f82
Successfully built bchlib


In [ ]:

import bchlib
# create a bch object
BCH_POLYNOMIAL = 8219
BCH_BITS = 72
bch = bchlib.BCH(BCH_POLYNOMIAL, BCH_BITS)
# v_encoded = bch.encode(V1.flatten())
# print(v_encoded)

In [ ]:

from reedsolo import RSCodec

# encode feature vector V using RS code
rs = RSCodec(10)
# V_encoded = rs.encode(V1.flatten())
# print(V_encoded)

In [ ]:
# encrypt image using XOR operation
img = Image.open(os.path.join(image_folder, '1-556.JPG')).convert('L') # load example image
img = img.resize((512, 512)) # resize image to 64x64 pixels
X = np.array(img).flatten() # convert image to numpy array
X_normalized = scaler.transform(X.reshape(1, -1)) # normalize image

V2 = model.predict(X_normalized) # extract feature vector V2
# bch ---> rs
V_encoded = bch.encode(V2.flatten()) # encode feature vector V2 using bch
v_encoded = rs.encode(np.frombuffer(V_encoded, dtype=np.uint8)) # convert bytearray to numpy array and encode using RS code
key = V_encoded
print(key)

1/1 [==============================] - 0s 284ms/step
bytearray(b"\x0fE\xd0\x03A\xe8\x9fM\x1bC\x06\xf0\xceR\x8e\xfa\x95~\x12A\xb8\x9aB\x8bnX\xb4r\xd2\xc2j\x1b\x9c\xed{E\x14\xef\xcdu\xb56\xa8\xcb\xfb\x1f\xebI\xe8\x15\xab\'\x8b$\xd1v\x11 \x8c \x8d&X><<\x1ez\x1d\xd3n\xfc\xa8\x91B]\x957\xee\x02\x15\xfa~\x01\x96\xd3\n\xf7\x96/\xab?\x8728\xe9\x00\x1a\x7f\x1c6\x14(\x8a\xf9\x84\x9f\x96j\xbd\x11\xfe\xc5h\x82\xc0\x00")


In [ ]:
import numpy as np
from scipy.stats import chisquare
import math
import matplotlib.pyplot as plt
from scipy import stats
# converting key into binary
binary_key = ''.join(format(byte, '08b') for byte in key)
print(binary_key)


111001011010001111101011011010000100101011111111110100010011000100110001010010011111011101000100000011100000000101010101011110100101001111000111100011111010010010001000000111110010001110011010010001001011000011001110101110000110110010010111100010101010011110001011111101001001000000100011010001001001111110001111011010011110110011001010001011011010111101100010100001000110101001000000001010111010101101101011100011011010111001010101100000010011110111010001000100110010010100000111100010001010011111001110111110111000000111010100011111011110111000101010110111001000000001111010101101110011000101101100110000000010010100010011101011110000111011111100100101110001010111000101000101111011000000111000101010001110010000111000010111000111001001101000110110010101011001110100010110011011000010011010111011000111100101001001011111011010010000101011010011010110011110100111111011101010110010111011011110000100001101100000000011000010000000000000


In [ ]:
# img = Image.open(os.path.join(image_folder, '1-556.JPG')).convert('L') # load example image
# img = img.resize((512, 512)) # resize image to 64x64 pixels
# X = np.array(img).flatten() # convert image to numpy array

# reshape key to match the shape of X
key_reshaped = np.tile(key, len(X) // len(key) + 1)[:len(X)]

# encrypt the image using XOR operation
X_encrypted = bytearray(X ^ key_reshaped)
save_encrypted_img_folder = "/content/gdrive/MyDrive/Encrypted_Images"
# save the encrypted image
img_encrypted = Image.frombytes('L', (512, 512), bytes(X_encrypted))
img_encrypted.save(os.path.join(save_encrypted_img_folder, 'BCH.jpg'))

In [ ]:
import cv2
save_encrypted_img_folder = "/content/gdrive/MyDrive/Encrypted_Images"
encrypted_image1 = np.array(Image.open(os.path.join(save_encrypted_img_folder, 'RS.jpg')))
encrypted_image2 = np.array(Image.open(os.path.join(save_encrypted_img_folder, 'BCH.jpg')))

# Compute the absolute difference between the two images
diff = cv2.absdiff(encrypted_image1, encrypted_image2)

# Compute the mean intensity value of the difference image
mean_intensity = np.sum(diff) / (diff.shape[0] * diff.shape[1])

# Compute the UACI value as a percentage
uaci = (mean_intensity / 255) * 100

# Print the UACI value
print("UACI is: %.2f%%" % uaci)
# bch ---> rs # UACI is: 33.24%
# rs ---> bch #UACI is:31.48%

In [ ]:
import numpy as np
from PIL import Image
image_folder = '/content/gdrive/MyDrive/iris/train/eye'
save_encrypted_img_folder = "/content/gdrive/MyDrive/Encrypted_Images"
# Load the original and encrypted images
original_image = Image.open(os.path.join(image_folder, '1-556.JPG'))
encrypted_image = Image.open(os.path.join(save_encrypted_img_folder, 'BCH+RS.jpg'))
original_image = original_image.resize((512, 512))
# Convert the images to grayscale
original_image = original_image.convert('L')
encrypted_image = encrypted_image.convert('L')

# Convert the images to numpy arrays for faster processing
original_array = np.array(original_image)
encrypted_array = np.array(encrypted_image)

# Calculate the number of pixels in the image
N = original_array.size

# Calculate the number of pixels that have changed between the original and encrypted images
diff_array = np.abs(original_array - encrypted_array)
Nc = np.count_nonzero(diff_array)

# Calculate NPCR
NPCR = (Nc/N) * 100

print('NPCR: {:.2f}%'.format(NPCR))
# bch->99.78
# RS ->99.61

NPCR: 99.70%


In [ ]:
# load the encrypted image
save_encrypted_img_folder = "/content/gdrive/MyDrive/Encrypted_Images"
img_encrypted = Image.open(os.path.join(save_encrypted_img_folder, 'BCH.jpg')).convert('L')

# convert the encrypted image to numpy array
X_encrypted = np.array(img_encrypted).flatten()

# reshape the key to match the shape of X_encrypted
key_reshaped = np.tile(key, len(X_encrypted) // len(key) + 1)[:len(X_encrypted)]

# use the decryption key to perform XOR operation on the encrypted image
X_decrypted = bytearray(X_encrypted ^ key_reshaped)

# convert the decrypted image to numpy array and reshape it
X_decrypted = np.array(X_decrypted)
X_decrypted = X_decrypted.reshape((512, 512))
save_decrypted_img_folder = "/content/gdrive/MyDrive/Decrypted_Images"
# convert the numpy array to an image and save it
img_decrypted = Image.fromarray(X_decrypted)
img_decrypted.save(os.path.join(save_decrypted_img_folder, 'BCH.jpg'))



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
image_folder = '/content/gdrive/MyDrive/iris/train/eye'
# Load the plain image
plain_img = Image.open(os.path.join(image_folder, '1-543.JPG'))

# Convert the plain image to a numpy array
plain_arr = np.array(plain_img)

# Draw the histogram for the plain image
plt.hist(plain_arr.ravel(), bins=256, range=(0, 255))
plt.title("Histogram of Plain iris Image")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.show()

# Load the encrypted image
encrypted_img = Image.open(os.path.join(image_folder, 'enc2.jpg'))

# Convert the encrypted image to a numpy array
encrypted_arr = np.array(encrypted_img)

# Draw the histogram for the encrypted image
plt.hist(encrypted_arr.ravel(), bins=256, range=(0, 255))
plt.title("Histogram of Encrypted Image")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.show()
